In [1]:
"""My suggestion will be to try to generate sub-dataframes for each class, 
in these sub-dataframes each row has same structure and columns are all named. 
If you can do that, you then try to 
created some sort of mapping from column 
names to you class property names, and use that mapping to populate your data 
classes. If you can do that, most of the hard coded column numbers will be gone, 
and column order doesn't matter anymore."""

"My suggestion will be to try to generate sub-dataframes for each class, \nin these sub-dataframes each row has same structure and columns are all named. \nIf you can do that, you then try to \ncreated some sort of mapping from column \nnames to you class property names, and use that mapping to populate your data \nclasses. If you can do that, most of the hard coded column numbers will be gone, \nand column order doesn't matter anymore."

In [2]:
import pandas as pd 
import numpy as np
import os, shutil
import inspect

from tqdm import tqdm

from dataclasses import dataclass, field
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

## Importing Functions and Dataclass
import seronetDataclass as seroClass
import seronetFunctions as seroFxn

warnings = 0
warn = 0


In [3]:
# directory Information (**kawrgs)
PMID = "33408181"


# Registry
sheet_name = 'SeroNet Registry Template'
map_sheet = 'Registry Definitions'

box_base = "~/Library/CloudStorage/Box-Box/SeroNet Public Data"

BASE_DIR = seroFxn.get_box_dir(box_base, PMID)
print(BASE_DIR)

file = "registry" + PMID
df_path = os.path.join(BASE_DIR,'templated_data', file + ".xlsx")
df_path = "./registry_v0.99_PMID33408181_noComments.xlsm"

# dictionary info
registryToImmportDict_file = "./dictionary/registryToBasic.csv"

registryToImmportDict = pd.read_csv(registryToImmportDict_file,
                                    header=None, 
                                    index_col=0, 
                                    squeeze=True).to_dict()

# ImmPort Templates (link to web?)
PATH_basic_stdy_template = './template/basic_study_design.xlsx'
PATH_protocols = './template/protocols.xlsx'
PATH_experiments = './template/experiments.xlsx'
PATH_reagent = './template/reagents.Other.xlsx'
PATH_assessment = './template/assessments.xlsx'
PATH_subject_human = './template/subjectHuman.xlsx'
PATH_subject_organism = './template/subjectAnimals.xlsx'


# Automate output... 
OUT_DIR = os.path.join(BASE_DIR, 'ImmPort_templates') 
OUT_DIR = './TEST'
PATH_pmid_basic_stdy_template = f'./PMID{PMID}_study.xlsx'

BASIC_STUDY_TEMPLATE = f'PMID{PMID}_basic'
EXP_TEMPLATE = f'PMID{PMID}_experiments'
PROTOCOL_TEMPLATE = f'PMID{PMID}_protocol'
REAGENT_TEMPLATE = f'PMID{PMID}_reagent'
ASSESSMENT_TEMPLATE =  f'PMID{PMID}_assessment'
SUBJ_HUMAN_TEMPLATE =  f'PMID{PMID}_subject_human'
SUBJ_ORGANISM_TEMPLATE =  f'PMID{PMID}_subject_organism'

# Make Dir if it does not exist
try:
    os.mkdir(OUT_DIR)
except FileExistsError:
    print('directory already exists')
    pass

# import file
book = load_workbook(df_path)
registry = book[sheet_name]
registry.delete_cols(1)


/Users/liualg/Library/CloudStorage/Box-Box/SeroNet Public Data/CA260541_SARIOL/PMID_33408181
directory already exists


/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
# add automate validation -- talk to Mani and Elaine
#   - error reporting through classes 
# no empty cell 
# date format (datetime) yyyy-mm-dd
# Error reporting 


In [5]:
# Class names
class_names = ['study_pubmed', 'study', 'study_personnel', 'study_file',
               'study_link', 'study_categorization', 'study_design',
               'protocol', 'condition_or_disease', 'Intervention Agent',
               'study_details', 'arm_cohort', 'inclusion_exclusion',
               'subject', 'Subject Type: human', 'Subject Type: model organism',
               'immuneExposure', 'planned_visit', 'Assessment', 'Study Experiment Samples',
               'Study Experiments', 'Reagent per Experiment-Assay Type', 
               'Results'
              ]

In [6]:
sp = seroFxn.get_sections(registry, class_names)

### Main Loop

In [7]:
# Looping through each section in the Registy template
for section_number in range(len(sp)-1):
    temp_wb = Workbook()
    temp_ws = temp_wb.active

    #making a temp workbook to store each section. This will be turned into df
    for i in registry.iter_rows(values_only = True,
                                min_row = sp[section_number]+1,
                                max_row = sp[section_number+1]-1):
        temp_ws.append(i)
        
    max_row = temp_ws.max_row
    max_col = temp_ws.max_column
    seroFxn.remove_excess(temp_ws)
#     print("Rows:", max_row, "->", temp_ws.max_row)
#     print("Columns:", max_col, "->", temp_ws.max_column)
    
    df = pd.DataFrame(temp_ws.values)
    sub_section = registry.cell(row=sp[section_number], column = 1).value
#     print(sub_section)

    if sub_section == 'study':
        df = seroFxn.edit_df(df)
        
        STUDY = seroClass.study(
            df['Study Identifier'][1],
            df['Study Name'][1],
            df['Publication Title'][1],
            df['Study Objective'][1],
            df['Study Description'][1],
            df['Primary Institution Name'][1]
            )
    
    elif sub_section == 'study_pubmed':
        df = seroFxn.edit_df(df)
        
        STUDY_PUBMED = seroClass.study_pubmed(
            df['Pubmed ID'][1]
#             list(df['Keyword'])
        )
        
    elif sub_section == 'study_personnel':
        df = seroFxn.edit_df(df)

        STUDY_PERSONNEL = seroClass.study_personnel(
            df['Personnel ID'],
            df['Honorific'],
            df['Last Name'],
            df['First Name'],
            df['Suffixes'],
            df['Organization'],
            df['ORCID ID'],
            df['Email'],
            df['Title In Study'],
            df['Role In Study'],
            df['Site Name']
        )


    elif sub_section == 'study_file':
        df = seroFxn.edit_df(df)

        STUDY_FILE = seroClass.study_file(
            df['Study File Name'],
            df['Study File Description'],
            df['Study File Type']  
        )
        
    
    elif sub_section == 'study_link':
        df = seroFxn.edit_df(df)

        STUDY_LINK = seroClass.study_link(
            df['Link Name'],
            df['Value']
        )
             
        
    elif sub_section == 'study_categorization':
        df = seroFxn.edit_df(df)

        STUDY_CATEGORIZATION = seroClass.study_categorization(
            df['Research Focus'][1],
            df['Study Type'][1],
            df['Keywords'][1],
        )
    
    elif sub_section == 'study_design':
        df = seroFxn.edit_df(df)

        if (df.shape != (2,0)): # checking size. There has to be a better way to do this

            STUDY_DESIGN = seroClass.study_design(
                df['Clinical Study Design'],
                df['in silico Model Type']
            )
        else:
            study_des = study_design()

    elif sub_section == 'protocol':
        df = seroFxn.edit_df(df)

        PROTOCOLS = seroClass.protocols(
            df['Protocol ID'],
            df['Protocol File Name'],
            df['Protocol Name'],
            df['Protocol Description'],
            df['Protocol Type'],
        )
        
    elif sub_section == 'condition_or_disease':
        df = seroFxn.edit_df(df)

        COD = seroClass.condition_or_disease(
            list(df['Reported Health Condition*'])
        )
        
    elif sub_section == 'Intervention Agent':
        df = seroFxn.edit_df(df)

        INTERVENTION_AGENT = seroClass.Intervention_Agent(
            list(df['SARS-CoV-2 Vaccine Type*'])
        )

    elif sub_section == 'study_details':
        df = seroFxn.edit_df(df)

        STUDY_DETAILS = seroClass.study_details(
            df['Clinical Outcome Measure'][1],
            df['Enrollment Start Date'][1],
            df['Enrollment End Date'][1],
            df['Number of Study Subjects'][1],
            df['Age Unit'][1],
            df['Minimum Age'][1],
            df['Maximum Age'][1]
            )

    elif sub_section == 'arm_cohort':
        df = seroFxn.edit_df(df)

        AOC = seroClass.arm_or_cohort(
            df['Arm ID'],
            df['Arm Name'],
            df['Study Population Description'],
            df['Arm Type']
        )


    elif sub_section == 'inclusion_exclusion':
        df = seroFxn.edit_df(df)

        INCLUSION_EXCLUSION = seroClass.inclusion_exclusion(
            df['Inclusion ID'],
            df['Inclusion Criterion'],
            df['Inclusion Criterion Category']  
        )

    elif sub_section == 'subject':
        df = seroFxn.edit_df(df)

        SUBJECT = seroClass.subject(
            df['Subject ID'][1],
            df['Sex at Birth*'][1],
            df['Age Event'][1],
            df['Subject Phenotype'][1],
            df['Study Location'][1]
            )
        
    elif sub_section == 'Subject Type: human':
        df = seroFxn.edit_df(df)

        SUBJECT_HUMAN = seroClass.subject_type_human(
            df['Ethnicity'],
            df['Race'],
            df['Race Specify'],
            df['Description']
            )

    elif sub_section == 'Subject Type: model organism':
        df = seroFxn.edit_df(df)

        SUBJECT_ORGANISM = seroClass.subject_type_mode_organism(
            df['Species'],
            df['Biosample Types'],
            df['Strain Characteristics']
            )
        
    elif sub_section == 'immuneExposure':
        df = seroFxn.edit_df(df)

        IMMUNEEXPOSURE = seroClass.immuneExposure(
            df['SARS-CoV2 History'][1],
            df['SARS-CoV-2 Vaccine Type'][1],
            df['COVID-19 Disease Severity'][1]
            )
    
    elif sub_section == 'planned_visit':
        df = seroFxn.edit_df(df)

        PLANNED_VISIT = seroClass.planned_visit(
            df['Visit ID'],
            df['Visit Name'],
            df['Visit Order Number'],
            df['Visit Min Start Day'],
            df['Visit Max Start Day'],
            df['Visit Start Rule']                                
    )
    
    elif sub_section == 'Assessment':
        df = seroFxn.edit_df(df)

        ASSESSMENT = seroClass.assessment(
            df['Assessment ID'],
            df['Assessment Name'],
            df['Measured Behavioral or Psychological Factor'],
            df['Measured Social Factor'],
            df['SARS-CoV-2 Symptoms'],
            df['Assessment_Clinical  and Demographic Data Provenance'],
            df['Assessment_Demographic Data Types Collected']                                
    )
    
    elif sub_section == 'Study Experiment Samples':
        df = seroFxn.edit_df(df)

        EXPERIMENT_SAMPLES = seroClass.study_experiment_samples(
            df['Expt Sample User Defined ID'],
            df['Expt Sample Biospecimen Type'],
            df['Expt Sample Biospecimen Collection Point']                             
    )
    
    elif sub_section == 'Study Experiments':
        df = seroFxn.edit_df(df)

        STUDY_EXPERIMENTS = seroClass.study_experiment(
            df['Experiment ID'],
            df['Experiment Name'],
            df['Experiment Assay Type'],
            df['Experiment Results File Name']
    )
        
    elif sub_section == 'Reagent per Experiment-Assay Type':
        df = seroFxn.edit_df(df)

        REAGENTS = seroClass.reagent_per_experiment(
            df['Reagent ID'],
            df['SARS-CoV-2 Antigen'],
            df['Assay Use'],
            df['Manufacturer'],
            df['Catalog #']
    )  
        
    elif sub_section == 'Results':
        df = seroFxn.edit_df(df)

        RESULTS = seroClass.results(
            df['Results Virus Target'],
            df['Results Antibody Isotype'],
            df['Results Reporting Units'],
            df['Results Reporting Format']
    )  
        
    else:
        print(sub_section, ': does not exist')


In [8]:
STUDY_CATEGORIZATION.Keywords

'Antibodies, NeutralizingAntibodies, ViralSpike Glycoprotein, Coronavirusspike protein, SARS-CoV-2'

### I need to add testing for correct values ###

- characters limit... but it also looks within limit
- datetime
- PMIDXXXXX_file-01
- check for pre-uploaded data


# Sending back to ImmPort Template #

In [9]:
# TO DO:
"""
Intervention Agent: needs to have a value if not NA
Protocol ID: Protocol ID.txt??
arm_or_cohort: is missing LOL (elaine)
inclusion_exclusion: needs data ... (elaine)

"""


'\nIntervention Agent: needs to have a value if not NA\nProtocol ID: Protocol ID.txt??\narm_or_cohort: is missing LOL (elaine)\ninclusion_exclusion: needs data ... (elaine)\n\n'

In [10]:
ImmPortClassNames = ['study', 'study_categorization', 'study_2_condition_or_disease',
               'arm_or_cohort','study_personnel', 'planned_visit', 'inclusion_exclusion',
               'study_2_protocol', 'study_file', 'study_link', 'study_pubmed'] 

In [11]:
shutil.copy(PATH_basic_stdy_template, PATH_pmid_basic_stdy_template)
basic_stdy_template = load_workbook(PATH_pmid_basic_stdy_template)
bst_ws = basic_stdy_template['basic_study_design.txt']

se = seroFxn.get_sections(bst_ws, ImmPortClassNames)


/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Basic Study Template ##

In [12]:
temp_wb = Workbook()
temp_ws = temp_wb.active

#making a temp workbook to store each section. This will be turned into df

# This is the first part of the df
for i in bst_ws.iter_rows(values_only = True,
                          max_row = se[2]+1):
    temp_ws.append(i)
    
# Starting at Arm or Cohort  
seroFxn.add_df(temp_ws, AOC)
seroFxn.add_df(temp_ws, STUDY_PERSONNEL)
seroFxn.add_df(temp_ws, PLANNED_VISIT)
seroFxn.add_df(temp_ws, INCLUSION_EXCLUSION)

# # Add protocol
temp_ws.append([])
temp_ws.append([PROTOCOLS.ImmPortNAME])
temp_ws.append(['Protocol ID', PROTOCOLS.Protocol_Name[1]])

seroFxn.add_df(temp_ws, STUDY_FILE)
seroFxn.add_df(temp_ws, STUDY_LINK)

seroFxn.add_df(temp_ws, STUDY_PUBMED)

pd.DataFrame(temp_ws.values).to_csv('./temp_mar6.csv', index=None)

#### Filling in Study Info Sections 

In [40]:
registryToImmportDict = pd.read_csv(registryToImmportDict_file, 
                                    header=None, 
                                    index_col=0, 
                                    squeeze=True).to_dict()

# registryToImmportDict
registryDict = {**vars(STUDY),
                **vars(STUDY_CATEGORIZATION),
                **vars(STUDY_DESIGN),
                **vars(STUDY_DETAILS),
                **vars(STUDY_PUBMED),
                **vars(IMMUNEEXPOSURE),
                **vars(COD)}

# Looping through ImmPort Template to get the correct order of the 'study' section
for se_number in range(se[0],se[3]):    
 
    if temp_ws["A"][se_number].value != None and registryToImmportDict.get(temp_ws["A"][se_number].value) != None:
#         print(temp_ws["A"][se_number].value)
        
        # Using a mapping key + using info in our classes to map the data
        print(temp_ws["A"][se_number].value)
        reg_key = registryToImmportDict.get(temp_ws["A"][se_number].value).strip().replace(' ',"_").replace('*',"")
#         print(registryDict.get(reg_key))
        try:
            # If input is a list, we will turn it into a string (since it can be a list)
            if type(registryDict.get(reg_key)) == list:
                temp_ws["B"][se_number].value = ', '.join(registryDict.get(reg_key))
            else:
                temp_ws["B"][se_number].value = registryDict.get(reg_key.replace('-',"_"))

        except:
            print(f"{reg_key} did not work")
            
#     else:
#         print('blank')


User Defined ID
Brief Title
Official Title
Brief Description
Description
Intervention Agent
Endpoints
Sponsoring Organization
Age Unit
Actual Start Date
Hypothesis
Objectives
Target Enrollment
Minimum Age
Maximum Age
Research Focus
Condition Reported


In [41]:
bsd = pd.DataFrame(temp_ws.values).replace({None: '', 'None': ''})
bsd

,0,1,2,3,4,5,6,7,8,9,10
0,basic_study_design,Schema Version 3.34,,,,,,,,,
1,Please do not delete or edit this column,The basic study design template defines and an...,,,,,,,,,
2,Column Name,,,,,,,,,,
3,,,,,,,,,,,
4,study,,,,,,,,,,
5,User Defined ID,PMID33408181,,,,,,,,,
6,Brief Title,Immunological memory to SARS-CoV-2 assessed fo...,,,,,,,,,
7,Official Title,Immunological memory to SARS-CoV-2 assessed fo...,,,,,,,,,
8,Brief Description,Understanding immune memory to SARS-CoV-2 has ...,,,,,,,,,
9,Description,INTRODUCTION: Immunological memory is the basi...,,,,,,,,,


In [16]:
# bsd.to_excel(os.path.join(OUT_DIR, f'PMID{PMID}_study.xlsx'), index=False, header = False)
bsd.to_csv(os.path.join(OUT_DIR,f'{BASIC_STUDY_TEMPLATE}.txt'),
           header = False,
           index = False,
           sep = '\t')

#### ERRORS
- does not like n/a, esp for dates

## Protocol ##

In [17]:
protocol_ws = load_workbook(PATH_protocols)['protocols.txt']
protocol_ws = seroFxn.remove_excess(protocol_ws)
seroFxn.add_df(protocol_ws, PROTOCOLS, add_header = False, stagger = 1)

In [18]:
protocol_df = pd.DataFrame(protocol_ws.values).replace({None: '', 'None': ''})
protocol_df.to_csv(os.path.join(OUT_DIR,f'{PROTOCOL_TEMPLATE}.txt'),
                   header = False, 
                   index = False,
                   sep = '\t')
# try:
#     shutil.copy(os.path.join(BASE_DIR,'templated_data', f'{PROTOCOLS.Name[1]}.txt'),
#                 os.path.join(OUT_DIR,f'{PROTOCOLS.Name[1]}.txt'))
# except:
#     print(f'File: {PROTOCOLS.Name[1]}.txt does not exist')

In [19]:
protocol_df

,0,1,2,3,4,5
0,protocols,Schema Version 3.34,,,,
1,Please do not delete or edit this column,,,,,
2,Column Name,User Defined ID,File Name,Name,Description,Type
3,,PMID33408181_protocol-01,PMID33408181_protocol-01.txt,PMID33408181_protocol-01,,Not Specified


#### Errors: NONE


## Experiment ##

In [20]:
# creating a map of the assay types to the SeroNet descriptors 
reg_description = pd.read_excel(df_path, sheet_name = map_sheet)
descriptions = dict(zip(reg_description['Unnamed: 1'][4:], reg_description['Unnamed: 2'][4:]))

#creating a df to add into the worksheet
Assay_used = STUDY_EXPERIMENTS.Experiment_Assay_Type
experiments_df = pd.DataFrame({
    'Column Name': ['']*len(Assay_used),
    'User Defined ID': [f'PMID{PMID}_exp-0'+str(i+1) for i,k in enumerate(Assay_used)],
    'Name': [Assay_used[i+1] for i, k in enumerate(Assay_used)],
    'Description': [descriptions.get(k) for i, k in enumerate(Assay_used)],
    'Measurement Technique': [STUDY_EXPERIMENTS.Experiment_Assay_Type[i+1] for i, k in enumerate(Assay_used)],
    'Study ID': [STUDY.Study_Identifier]*len(Assay_used),
    'Protocol ID(s)': [PROTOCOLS.Protocol_ID[1]]*len(Assay_used)
})

In [21]:
# loading experiment template and removing excess rows and columns
experiment_ws = load_workbook(PATH_experiments)['experiments.txt']
experiment_ws = seroFxn.remove_excess(experiment_ws)

# adding df to bottom of ws
seroFxn.add_df(experiment_ws, experiments_df, add_header = False)


/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [22]:
experiments_df = pd.DataFrame(experiment_ws.values).replace({None: '', 'None': ''})
experiments_df.to_csv(os.path.join(OUT_DIR,f'{EXP_TEMPLATE}.txt'),
                      header = False, 
                      index = False,
                      sep = '\t')

experiments_df

,0,1,2,3,4,5,6
0,experiments,Schema Version 3.34,,,,,
1,Please do not delete or edit this column,,,,,,
2,Column Name,User Defined ID,Name,Description,Measurement Technique,Study ID,Protocol ID(s)
3,,PMID33408181_exp-01,PCR,Study has reported results from PCR assays.,PCR,PMID33408181,PMID33408181_protocol-01


#### errors: NONE

### ASSESSMENT ###

In [23]:
NumAssessments = len(ASSESSMENT.Assessment_ID)
empty = [''] * NumAssessments
assessment_name = []

for i in range(NumAssessments): ### THIS NEEDS TO BE 1 cell to the right. So the actual vaules 
    i += 1
    if ASSESSMENT.Measured_Behavioral_or_Psychological_Factor[i]:
        # 'Measured Behavioral or Psychological Factor'
        assessment_name.append(ASSESSMENT.Measured_Behavioral_or_Psychological_Factor[i])
    elif ASSESSMENT.Measured_Social_Factor[i]: 
        # 'Measured Social Factor'
        assessment_name.append(ASSESSMENT.Measured_Social_Factor[i])
    elif ASSESSMENT.SARS_CoV_2_Symptoms[i]:
        # 'SARS-CoV-2 Symptoms'
        assessment_name.append(ASSESSMENT.SARS_CoV_2_Symptoms[i])
    else:
        print('No assessment name')


# need to figure out named report 
# not sure where the data goes for the assessments

assessment_df = pd.DataFrame({
    'Column Name': empty,
    'Subject ID': [SUBJECT.Subject_ID] * NumAssessments,  #assuming 1 subj per study .. treat as a study id
    'Assessment Panel ID': ASSESSMENT.Assessment_ID,
    'Study ID': [STUDY.Study_Identifier] * NumAssessments,
    'Name Reported': assessment_name,
    'Assessment Type': ASSESSMENT.Assessment_Clinical_and_Demographic_Data_Provenance,
    'Status': ASSESSMENT.Assessment_Demographic_Data_Types_Collected,
    'CRF File Names': empty,
    'Result Separator Column': empty,
    'User Defined ID': ASSESSMENT.Assessment_ID, #not sure how to map this, map say assesment compontnet ID
    'Planned Visit ID': PLANNED_VISIT.User_Defined_ID,
    'Name Reported': assessment_name, # Not sure, this says assessment component name
    'Study Day': empty, # Not sure, this says assessment component study day
    'Age At Onset Reported': empty,
    'Age At Onset Unit Reported': empty,
    'Is Clinically Significant': empty,
    'Location Of Finding Reported': empty,
    'Organ Or Body System Reported': empty,
    'Result Value Reported': empty,
    'Result Unit Reported': empty,
    'Result Value Category': empty,
    'Subject Position Reported': empty,
    'Time Of Day': empty,
    'Verbatim Question': empty,
    'Who Is Assessed': empty
})

In [24]:
# loading experiment template and removing excess rows and columns
assessment_ws = load_workbook(PATH_assessment)['assessments.txt']
assessment_ws = seroFxn.remove_excess(assessment_ws)

# adding df to bottom of ws
seroFxn.add_df(assessment_ws, assessment_df, add_header = False)
assessment_df = pd.DataFrame(assessment_ws.values).replace({None: '', 'None': ''})

# saving file
assessment_df.to_csv(os.path.join(OUT_DIR,f'{ASSESSMENT_TEMPLATE}.txt'),
                   header = False, 
                   index = False,
                   sep = '\t')
assessment_df

/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,assessments,Schema Version 3.34,,,,,,,,,...,,,,,,,,,,
1,Please do not delete or edit this column,,,,,,,,,,...,,,,,,,,,,
2,Column Name,Subject ID,Assessment Panel ID,Study ID,Name Reported,Assessment Type,Status,CRF File Names,Result Separator Column,User Defined ID,...,Is Clinically Significant,Location Of Finding Reported,Organ Or Body System Reported,Result Value Reported,Result Unit Reported,Result Value Category,Subject Position Reported,Time Of Day,Verbatim Question,Who Is Assessed
3,,PMID33408181_subject-01,PMID33408181_assess-01,PMID33408181,Post-acute sequelae of COVID-19 (>= 4 weeks),From Electronic Health Record,n/a,,,PMID33408181_assess-01,...,,,,,,,,,,
4,,PMID33408181_subject-01,PMID33408181_assess-02,PMID33408181,Vaccine Confidence,Self Reported,n/a,,,PMID33408181_assess-02,...,,,,,,,,,,


#### ERRORS


### REAGENT ###

In [25]:
NumReagents = len(REAGENTS.Reagent_ID)
empty = [''] * NumReagents

reagent_df = pd.DataFrame({
    'Column Name': empty,
    'User Defined ID': REAGENTS.Reagent_ID,
    'Name': REAGENTS.SARS_CoV_2_Antigen,
    'Description': REAGENTS.Assay_Use,
    'Manufacturer': REAGENTS.Manufacturer,
    'Catalog Number': REAGENTS.Catalog,
    'Lot Number': empty,
    'Weblink': empty,
    'Contact': empty
    
})

In [26]:
# loading experiment template and removing excess rows and columns
reagent_ws = load_workbook(PATH_reagent)['reagents.Other.txt']
reagent_ws = seroFxn.remove_excess(reagent_ws)

# adding df to bottom of ws
seroFxn.add_df(reagent_ws, reagent_df, add_header = False)
reagent_df = pd.DataFrame(reagent_ws.values).replace({None: '', 'None': ''})

# saving df
reagent_df.to_csv(os.path.join(OUT_DIR,f'{REAGENT_TEMPLATE}.txt'),
                   header = False, 
                   index = False,
                   sep = '\t')

reagent_df

/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,0,1,2,3,4,5,6,7,8
0,other_reagents,Schema Version 3.34,,,,,,,
1,Please do not delete or edit this column,,,,,,,,
2,Column Name,User Defined ID,Name,Description,Manufacturer,Catalog Number,Lot Number,Weblink,Contact
3,,PMID33408181_reagent-01,Receptor Binding Domain (RBD),n/a,n/a,n/a,,,


### SUBJECTS

- if human, use human
- if animal, use animal sheet 
- use length as a predictor
- THIS WILL CAUSE ME PROBLEMS> WILL NEED TO TEST THIS

In [27]:
if SUBJECT_HUMAN:
    print ("SUBJECT_human data")
    species = [SUBJECT.Subject_ID]
    empty = ['']*len(species)
    SUBJECT_human_df = pd.DataFrame({
        'Column Name': empty,
        'Subject ID': SUBJECT.Subject_ID,
        'Arm Or Cohort ID': AOC.User_Defined_ID,
        'Gender': SUBJECT.Sex_at_Birth, 
        'Min Subject Age': empty,
        'Max Subject Age': empty,
        'Age Unit': [STUDY_DETAILS.Age_Unit]*len(species),
        'Age Event': SUBJECT.Age_Event, 
        'Age Event Specify': empty,
        'Subject Phenotype': empty, 
        'Subject Location': SUBJECT.Study_Location, 
        'Race': SUBJECT_HUMAN.Race, 
        'Race Specify': SUBJECT_HUMAN.Race_Specify, 
        'Description': empty, 
        'Result Separator Column': empty, 
        'Exposure Process Reported': empty,
        'Exposure Material Reported': empty,
        'Exposure Material ID': empty,
        'Disease Reported': empty,
        'Disease Ontology ID': empty,
        'Disease Stage Reported': empty 
    })
    
    # loading experiment template and removing excess rows and columns
    human_ws = load_workbook(PATH_subject_human)['subjectHumans.txt']
    human_ws = seroFxn.remove_excess(human_ws)

    # adding df to bottom of ws
    seroFxn.add_df(human_ws, SUBJECT_human_df, add_header = False)
    SUBJECT_human_df = pd.DataFrame(human_ws.values).replace({None: '', 'None': ''})

    # saving df
    SUBJECT_human_df.to_csv(os.path.join(OUT_DIR,f'{SUBJ_HUMAN_TEMPLATE}.txt'),
                       header = False, 
                       index = False,
                       sep = '\t')

    SUBJECT_human_df.head()


SUBJECT_human data


In [28]:
if SUBJECT_ORGANISM:  # Not sure how this plays out. Might need to do a mock one. Will it be 1 subject per study?
    print ("SUBJECT_organism data")
    species = [SUBJECT.Subject_ID]
    empty = ['']*len(species)
    SUBJECT_organism_df = pd.DataFrame({
        'Column Name':empty,
        'Subject ID': SUBJECT.Subject_ID, 
        'Arm Or Cohort ID': empty, # I feel like this needs to be inputted
        'Gender': SUBJECT.Sex_at_Birth,
        'Min Subject Age': empty,
        'Max Subject Age': empty,
        'Age Unit': [STUDY_DETAILS.Age_Unit] * len(species),
        'Age Event': SUBJECT.Age_Event,
        'Age Event Specify': empty,
        'Subject Phenotype': empty, ## This does not exist
        'Subject Location': SUBJECT.Study_Location,
        'Species': SUBJECT_ORGANISM.Species,
        'Strain': SUBJECT_ORGANISM.Biosample_Types,
        'Strain Characteristics': SUBJECT_ORGANISM.Strain_Characteristics,
        'Result Separator Column': empty,
        'Exposure Process Reported': IMMUNEEXPOSURE.SARS_CoV2_History,
        'Exposure Material Reported': IMMUNEEXPOSURE.SARS_CoV_2_Vaccine_Type,
        'Exposure Material ID': [i.split(';')[1] for i in IMMUNEEXPOSURE.SARS_CoV_2_Vaccine_Type],  # Should this Vaccine ID. Need to search this
        'Disease Reported': COD.Reported_Health_Condition, # also maybe not...
        'Disease Ontology ID': empty,
        'Disease Stage Reported': IMMUNEEXPOSURE.COVID_19_Disease_Severity
    })
    
        # loading experiment template and removing excess rows and columns
    organism_ws = load_workbook(PATH_subject_organism)['subjectAnimals.txt']
    organism_ws = seroFxn.remove_excess(organism_ws)

    # adding df to bottom of ws
    seroFxn.add_df(organism_ws, SUBJECT_organism_df, add_header = False)
    SUBJECT_organism_df = pd.DataFrame(organism_ws.values).replace({None: '', 'None': ''})

    # saving df
    SUBJECT_organism_df.to_csv(os.path.join(OUT_DIR,f'{SUBJ_ORGANISM_TEMPLATE}.txt'),
                       header = False, 
                       index = False,
                       sep = '\t')

    SUBJECT_organism_df
    
#     subjectAnimals
else:
    print("no data")

SUBJECT_organism data


IndexError: list index out of range

In [ ]:
[i.split(';')[1] for i in IMMUNEEXPOSURE.SARS_CoV_2_Vaccine_Type]

In [ ]:
COD.Reported_Health_Condition

In [ ]:
reagent_df.to_csv(os.path.join(OUT_DIR,f'{REAGENT_TEMPLATE}.txt'),
                   header = False, 
                   index = False,
                   sep = '\t')

In [ ]:
if SUBJECT_HUMAN:
    print ("Has Data")
else:
    print ("No Data")

In [ ]:
class my_class(object):
    def __init__(self):
        self.lis1 = []
        self.dict1 = []

#     def __nonzero__(self):
#         return bool(self.lis1 or self.dict1)
    
    def __len__(self):
        for field in self.fields():
            print(field.name, getattr(self, field.name))
            
        return len(self.lis1) + len(self.dict1)
    
    def __post_init__(self):
        for field in self.__dataclass_fields__:
            value = getattr(self, field)
            print(field, value)

In [ ]:
obj = my_class()
# obj.lis1 = [1,2,3,4]
# obj.dict1 = [1,2,3,4]

if obj:
    print ("Available")
else:
    print ("Not available")